In [ ]:
import json
from typing import TypedDict, List
from typing_extensions import Annotated, Literal
from pydantic import BaseModel, Field

import numpy as np
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
class Action(BaseModel):
    """
    A model that defines an action for the robot to execute.
    """
    action: Literal["move"] = Field(
        None, description="The action for the robot to execute. Must be 'move'."
    )
    start: str = Field(
        None, description="The starting position/location for the robot's action."
    )
    end: str = Field(
        None, description="The ending position/location for the robot's action."
    )

In [ ]:
class Plan(BaseModel):
    """
    A plan consisting of sequential actions for the robot to complete a mission.
    """
    actions: List[Action] = Field(
        None, description="A list of actions for the robot to execute."
    )

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")
# LLM에거 구조화된 답변을 생성하도록 함
structured_llm = llm.with_structured_output(Plan)

In [ ]:
cube = [
    {"name": "red-1", "color": "red", "x": 10, "y": 10},
    {"name": "red-2", "color": "red", "x": 100, "y": 100},
    {"name": "red-3", "color": "red", "x": 10, "y": 90},
    {"name": "red-4", "color": "red", "x": 90, "y": 10},
    {"name": "red-5", "color": "red", "x": 50, "y": 50},
    {"name": "blue-1", "color": "blue", "x": 100, "y": 10},
    {"name": "blue-2", "color": "blue", "x": 10, "y": 100},
    {"name": "blue-3", "color": "blue", "x": 100, "y": 20},
    {"name": "blue-4", "color": "blue", "x": 20, "y": 100},
    {"name": "blue-5", "color": "blue", "x": 40, "y": 40}
]

In [ ]:
template_ko = """당신은 가정내의 간단한 임무를 수행하는 로봇입니다.
아래 명령은 예시로서 "red-1" 물건을 "red-2" 지점으로 옮기는 명령입니다.
{{
    {{"action": "move", "start": "red-1", "end": "red-2"}}
}}

현재 물건이 아래와 같이 큐브가 놓여 있습니다.
{cube_state}

이 큐브를 색깔별로 모으려고 합니다.
이 큐브를 모으기 위한 적당한 명령을 생성해 주세요.
"""

prompt_template_ko = PromptTemplate(
    template=template_ko,
    input_variables=["cube_state"]
)

print(prompt_template_ko.format(cube_state=json.dumps(cube, indent=4)))

In [ ]:
result = structured_llm.invoke(prompt_template_ko.format(cube_state=json.dumps(cube, indent=4)))

for action in result.actions:
    print(action)

In [ ]:
template_en = """You are a household robot command generator. Your task is to analyze object positions and generate optimal movement commands to group objects by color.

COMMAND FORMAT:
{{
    {{"action": "move", "start": "red-1", "end": "red-2"}}
}}

RULES:
1. Analyze the current positions of all objects
2. Group objects by color at the most central or strategic location
3. Minimize total movement distance
4. Generate commands in logical sequence
5. Only move objects to positions of other objects with the same color

CURRENT OBJECT POSITIONS:
{cube_state}

TASK: Generate the minimum number of movement commands to group all cubes by color.

OUTPUT FORMAT: Provide a JSON array of commands with brief explanation of your strategy.
"""

prompt_template_en = PromptTemplate(
    template=template_en,
    input_variables=["cube_state"]
)

print(prompt_template_en.format(cube_state=json.dumps(cube, indent=4)))

In [ ]:
result = structured_llm.invoke(prompt_template_en.format(cube_state=json.dumps(cube, indent=4)))

for action in result.actions:
    print(action)

In [ ]:
cmd = 4
red = np.random.randint(0, 100, (cmd, 2))
blue = np.random.randint(0, 100, (cmd, 2))

plt.scatter(red[:, 0], red[:, 1], color="red")
plt.scatter(blue[:, 0], blue[:, 1], color="blue")
plt.show()

red_dict = {f"red-{i}":r for i, r in enumerate(red)}
blue_dict = {f"blue-{i}":r for i, r in enumerate(blue)}
all_dict = (red_dict | blue_dict)
keys = list(all_dict.keys())

cube = []
for key in np.random.permutation(keys):
    p = all_dict[key]
    cube.append({"name": str(key), "color": key.split("-")[0], "x": int(p[0]), "y": int(p[1])})
print(cube)

In [ ]:
result = structured_llm.invoke(prompt_template_en.format(cube_state=json.dumps(cube, indent=4)))

In [ ]:
for action in result.actions:
    start = all_dict[action.start]
    end = all_dict[action.end]
    start[0] = end[0] + np.random.randint(-5, 5)
    start[1] = end[1] + np.random.randint(-5, 5)

    print(action)
    plt.scatter(red[:, 0], red[:, 1], color="red")
    plt.scatter(blue[:, 0], blue[:, 1], color="blue")
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.show()